In [19]:
import spacy
import nltk
import gensim
from gensim import corpora
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
import time
from nltk.stem.wordnet import WordNetLemmatizer
from contextlib import contextmanager
import random
import csv
import pandas as pd

In [20]:
@contextmanager
def timer(msg):
    t0 = time.time()
    print(f'[{msg}] start.')
    yield
    elapsed_time = time.time() - t0
    print(f'[{msg}] done in {elapsed_time / 60:.2f} min.')

In [21]:
spacy.load('en')
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
#         elif token.like_url:
#             lda_tokens.append('URL')
#         elif token.orth_.startswith('@'):
#             lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens
nltk.download('wordnet')

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
additional_stops = ['&lt;', '&gt;', 'lt;#&gt', 'lt;', '&gt', 'know', 'just', 'txt', 'like', 'ok', 'come', 'want', 'did', 'got']

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
en_stop.update(additional_stops)
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
#     tokens = [get_lemma(token) for token in tokens]
    return tokens

In [24]:
data = pd.read_csv('cleaned_short.csv')

In [25]:
data.head()

,target,comment_text
0,0.4400,this is a great story man i wonder i...
1,0.9125,yet call out all muslims for the acts of a few...
2,0.0000,because the people who drive cars more are the...
3,0.3000,mormons have had a complicated relationship wi...
4,0.0000,i am doing the same thing


In [26]:
text_data = []

for row in data.comment_text:
    tokens = prepare_text_for_lda(row)
    text_data.append(tokens)

In [27]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

In [28]:
with timer('LDA'):
    NUM_TOPICS = 30
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=10, random_state=0)
#     ldamodel.save('model5.gensim')
    topics = ldamodel.print_topics()

[LDA] start.
[LDA] done in 18.00 min.


In [30]:
with timer('corpus'):
    lda_corpus = ldamodel[corpus]
    lda_corpus

[corpus] start.
[corpus] done in 0.00 min.


In [31]:
with timer('lda_docs'):
    lda_docs = [doc for doc in lda_corpus]

[lda_docs] start.
[lda_docs] done in 1.22 min.


In [34]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [36]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6      0.269656 -0.059748       1        1  9.786043
5      0.288766 -0.059024       2        1  8.438702
15     0.218746  0.108217       3        1  6.749079
17     0.258225 -0.043968       4        1  6.663603
10     0.151552  0.091964       5        1  6.207407
12     0.269685 -0.052398       6        1  5.495002
26     0.164777 -0.052549       7        1  5.279867
8      0.167233  0.008913       8        1  5.024935
28     0.095934  0.060417       9        1  3.717370
27     0.035502  0.073262      10        1  3.524749
29     0.008973 -0.158122      11        1  2.891443
4     -0.019009  0.143668      12        1  2.840584
23    -0.054494 -0.009012      13        1  2.477359
16    -0.043021 -0.128990      14        1  2.358483
25    -0.108635  0.171012      15        1  2.333656
22    -0.095289  0.160898      16        1  2.275684
11    -0.051134  0.014294      17        1  2.250400
19    -0.045122 -0.000695      18        1  2.154557
21    -0.098392  0.137931      19        1  2.093658
7     -0.056215  0.015444      20        1  2.054407
13    -0.082134  0.145031      21        1  1.851981
20    -0.121948  0.042294      22        1  1.768536
2     -0.146651 -0.043561      23        1  1.583786
9     -0.133800 -0.012185      24        1  1.569314
24    -0.146821 -0.000576      25        1  1.561963
0     -0.101769 -0.192628      26        1  1.501251
14    -0.156155 -0.087867      27        1  1.439127
3     -0.159426 -0.113689      28        1  1.423732
18    -0.157766 -0.081023      29        1  1.346271
1     -0.151268 -0.077310      30        1  1.337041, topic_info=     Category          Freq      Term         Total  loglift  logprob
391   Default  33329.000000     women  33329.000000  30.0000  30.0000
364   Default  25782.000000     white  25782.000000  29.0000  29.0000
2464  Default  30155.000000     trump  30155.000000  28.0000  28.0000
1293  Default  25666.000000    church  25666.000000  27.0000  27.0000
26    Default  57194.000000    people  57194.000000  26.0000  26.0000
...       ...           ...       ...           ...      ...      ...
512   Topic30   1000.257141  speaking   2123.148193   3.5621  -4.4257
1999  Topic30    989.616516     learn   2507.849365   3.3848  -4.4364
5458  Topic30    898.050110    source   2021.040283   3.5036  -4.5335
532   Topic30    998.714966      give   7154.805664   2.3456  -4.4273
423   Topic30    626.447510    status   1628.763916   3.3592  -4.8937

[1439 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
9183      4  0.997216    abiding
2439     12  0.999781   abortion
4350     12  0.998367  abortions
1097      9  0.215956      abuse
1097     12  0.783809      abuse
...     ...       ...        ...
9032     18  0.997644       york
1564      6  0.844810      young
1564     16  0.155105      young
315      25  0.999164    younger
1697     25  0.998987    youtube

[2528 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 16, 18, 11, 13, 27, 9, 29, 28, 30, 5, 24, 17, 26, 23, 12, 20, 22, 8, 14, 21, 3, 10, 25, 1, 15, 4, 19, 2])